# The experiment with learning rate shedulers 

In this notebook, we will compare different learning rate schedules. 
We'll train ResNet34 with learning rate schedules from `domain` using dataset Imagenette160 for training and testing. At the end of the notebook, there are plots with learning rates, losses, and accuracy on the test dataset during the whole training process for different learning rate schedules. Also, there is the table for more detailed comparison.

In [1]:
import sys

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

sys.path.append('../../..')

from batchflow import Pipeline, B, V, C
from batchflow.opensets import Imagenette160
from batchflow.models.torch import ResNet34
from batchflow.models.metrics import ClassificationMetrics
from batchflow.research import Research, Option, Results, KV, RP, REU, RI
from batchflow.utils import plot_results_by_config, show_research, print_results

Research parameters:

In [2]:
NUM_ITERS = 50000                               # number of iterations to train each model for
N_REPS = 7                                      # number of times to repeat each model train
RESEARCH_NAME = 'research'                      # name of Research object
DEVICES = [5, 6, 7]                             # devices to use
WORKERS = len(DEVICES)                          # number of simultaneously trained models
TEST_FREQUENCY = 150
BATCH_SIZE = 64

dataset = Imagenette160()                       # dataset to train models on
DS_TRAIN_LEN = len(dataset.train)

### Research domain

Learning rate schedules seek to adjust the learning rate during training by managing the learning rate according to a pre-defined schedule. Our domain consists of various learning rate decays and warmups. Parameters `'name'`, `'frequency'`, `'first_iter'`, `'last_iter'` are for BatchFlow, other parameters for chosen decay.
In a nutshell, about BatchFlow's parameters for decay:

- `'name'` - choose decay sheduler;
- `'frequency'` - how often the decay step is applied;
- `'first_iter'` - first iteration since which decay starts to work (default 0);
- `'last_iter'` - last iteration on which decay works (default ∞);

Let's check how useful learning rate shedulers are! 

In [3]:
domain = (Option('decay', [
    KV({'name': 'exp', 'gamma': .96, 'frequency': DS_TRAIN_LEN // BATCH_SIZE},
       'ordinary_1'),
    KV({'name': 'exp', 'gamma': .1, 'frequency': 7500},
       'ordinary_2'),
    KV([{'name': 'exp', 'gamma': 1.01, 'frequency': 6, 'last_iter': 900},
        {'name': 'exp', 'gamma': .9994, 'frequency': 2, 'first_iter': 901, 'last_iter': 22000}],
       'with_warmup_1'),
    KV([{'name': 'exp', 'gamma': 1, 'frequency': 1, 'last_iter': 900},
        {'name': 'exp', 'gamma': .9995, 'frequency': 2, 'first_iter': 901}],
       'with_warmup_2'),
    KV({'name': 'CyclicLR', 'base_lr': .0002, 'max_lr': .0016, 
        'step_size_up': 550, 'cycle_momentum': False, 
        'mode': 'exp_range', 'gamma': .99998, 'frequency': 1},
       'cyclic'),
    KV(None, 'None')
]))

Name and corresponding scheduler:

| 'name' | scheduler | 
|------|-------------|
|'exp' |[torch.optim.lr_scheduler.ExponentialLR](https://pytorch.org/docs/stable/optim.html?highlight=cyclic#torch.optim.lr_scheduler.ExponentialLR)|
|'CyclicLR'|[torch.optim.lr_scheduler.CyclicLR](https://pytorch.org/docs/stable/optim.html?highlight=cyclic#torch.optim.lr_scheduler.CyclicLR)|

Last option ```KV(None, 'None')``` - without a scheduler.

Default config for our ResNet34 model with 10 classes. Using named expression `C` we substitute chosen decay and device.

In [4]:
config = {
    'inputs/labels/classes': 10,
    'head/layout': 'Vf',
    'head/units': 10,
    'decay': C('decay'),
    'device': C('device'),
}

### Train pipeline

Root and branch pipelines for the models training. We save loss and learning rate values for further comparison.

In [5]:
train_root = (dataset.train.p      
                  .crop(shape=(160, 160), origin='center')
                  .to_array(channels='first', dtype=np.float32)
                  .multiply(multiplier=1/255)
                  .run_later(BATCH_SIZE, n_epochs=None, drop_last=True,
                             shuffle=True)
                   )

train_pipeline = (Pipeline()
                  .init_variable('loss')
                  .init_variable('learning_rate')
                  .init_model('dynamic', ResNet34, 'my_model', config=config) 
                  .train_model('my_model', B('images'), B('labels'), 
                               fetches=['loss', 'lr'], 
                               save_to=[V('loss'), V('learning_rate')]
                              )
                 )

### Test pipeline and accuracy calculation

Function for accuracy computation on the test dataset. We will use it for decay comparison.

In [6]:
def accuracy_func(iteration, import_from):
    pipeline = (dataset.test.p
                 .import_model('my_model', import_from)
                 .init_variable('metrics')
                 .init_variable('predictions')
                 .crop(shape=(160, 160), origin='center')
                 .to_array(channels='first', dtype=np.float32)
                 .multiply(multiplier=1/255)
                 .predict_model('my_model', B('images'), fetches='predictions',
                                save_to=V('predictions'))
                 .gather_metrics('class', targets=B.labels, predictions=V('predictions'),
                                 fmt='logits', axis=-1, save_to=V('metrics', mode='a'))
                 .run(128, n_epochs=1, drop_last=False, shuffle=True)
                 )
    metrics = test_pipeline.get_variable('metrics')
    return metrics.evaluate('accuracy')

### Research

Research object that combines all of the foregoing and launches experiments in parallel. Results for all models will be collected in one file.

In [7]:
research = (Research()
            .init_domain(domain, n_reps=N_REPS)
            .add_pipeline(root=train_root, branch=train_pipeline, variables=['loss', 'learning_rate'],
                          name='train_pipeline', logging=True)
            .add_callable(accuracy_func, returns='accuracy', name='accuracy_func', execute=TEST_FREQUENCY,
                          iteration=RI(), import_from=RP('train_pipeline')))

Start research!

In [ ]:
research.run(NUM_ITERS, name=RESEARCH_NAME,
             devices=DEVICES, workers=WORKERS,
             bar=True)

Research research is starting...


Domain updated: 0:   3%|▎         | 65512/2100000.0 [3:28:02<107:40:51,  5.25it/s]

### Results and conclusion

Load file with research results:

In [ ]:
results = Results(path=RESEARCH_NAME, concat_config=True)

Plots of the learning rate, loss, and accuracy on the test dataset.
The plots have curves for each model averaged over `N_REPS` executions.

In [12]:
show_research(results.df, average_repetitions=True,
              layout=['train_pipeline/learning_rate', 'train_pipeline/loss', 'accuracy_func/accuracy'],  
              color=list(mcolors.TABLEAU_COLORS.keys()), log_scale=False, rolling_window=10, 
              nrows=3, ncols=1, figsize=(20, 28))

The next table consists of columns of averaged test accuracy of last `n_last` measurements for each model for each execution. And last 2 columns in table is averaged accuracy over all executions  - `accuracy_func (mean)` and standard deviation - `accuracy_func (std)`

In [13]:
print_results(results.df, 'accuracy_func/accuracy', False, ascending=True, n_last=100)

,accuracy_func (exec 5),accuracy_func (exec 0),accuracy_func (exec 3),accuracy_func (exec 6),accuracy_func (exec 2),accuracy_func (exec 1),accuracy_func (exec 4),accuracy_func (mean),accuracy_func (std)
decay_None,0.815195,0.831622,0.848049,0.837782,0.831622,0.850103,0.848049,0.837489,0.011638
decay_cyclic,0.864476,0.866530,0.860370,0.868583,0.862423,0.870637,0.852156,0.863596,0.005688
decay_ordinary_1,0.860370,0.864476,0.850103,0.866530,0.862423,0.862423,0.856263,0.860370,0.005148
decay_ordinary_2,0.862423,0.852156,0.850103,0.862423,0.856263,0.856263,0.856263,0.856556,0.004311
decay_with_warmup_1,0.858316,0.852156,0.848049,0.856263,0.841889,0.856263,0.872690,0.855089,0.008907
decay_with_warmup_2,0.854209,0.839836,0.864476,0.856263,0.854209,0.862423,0.862423,0.856263,0.007761


We take a look at column `accuracy_func (mean)`.
From the values in that column, it is obvious that a learning rate schedule should be used, because of the result without learning rate decay is worst. 
And further on the most prospective learning rate schedules from that research: `'cyclic'` and `'ordinary_1'`. You can tune them and use them on your projects. But besides that, the current notebook can be your base notebook for testing other schedules for other tasks.